In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_excel('/content/telegram_data.xlsx')

In [4]:
df

,Channel Title,Channel Username,ID,Message,Date,Media Path
0,Sheger online-store,@Shageronlinestore,5333,NaN,2024-09-20 11:50:03+00:00,photos/@Shageronlinestore_5333.jpg
1,Sheger online-store,@Shageronlinestore,5332,NaN,2024-09-20 11:50:03+00:00,photos/@Shageronlinestore_5332.jpg
2,Sheger online-store,@Shageronlinestore,5331,NaN,2024-09-20 11:50:03+00:00,photos/@Shageronlinestore_5331.jpg
3,Sheger online-store,@Shageronlinestore,5330,NaN,2024-09-20 11:50:02+00:00,photos/@Shageronlinestore_5330.jpg
4,Sheger online-store,@Shageronlinestore,5329,NaN,2024-09-20 11:50:02+00:00,photos/@Shageronlinestore_5329.jpg
...,...,...,...,...,...,...
5010,Sheger online-store,@Shageronlinestore,10,🎯 3in1 One Step Hair Dryer & Styler \n\n👉 ከርል ...,2021-04-27 05:57:12+00:00,photos/@Shageronlinestore_10.jpg
5011,Sheger online-store,@Shageronlinestore,9,✅ Home GYM - X5 slimming vibrator \n\n📢📢📢 ታላቅ ...,2021-04-27 05:45:57+00:00,photos/@Shageronlinestore_9.jpg
5012,Sheger online-store,@Shageronlinestore,4,ለጤናችን-Health & Personal Care\n\n📍FingerTip Pul...,2021-04-12 08:36:40+00:00,photos/@Shageronlinestore_4.jpg
5013,Sheger online-store,@Shageronlinestore,3,#Finger_tip_pulse_oximeter\n #በተመጣጣኝ_ዋጋ\...,2021-04-12 08:35:47+00:00,photos/@Shageronlinestore_3.jpg


In [5]:
# Fill missing values in 'Message' with empty strings
df['Message'] = df['Message'].fillna('')

In [6]:
# remove other rows and keep only the message column
df = df[['Message']]

In [7]:
import re
# Define a function to remove emojis
def remove_emojis(text):
    if text is not None:  # Check if text is not None
        emoji_pattern = re.compile(
            "["
            "\U0001F600-\U0001F64F"  # emoticons
            "\U0001F300-\U0001F5FF"  # symbols & pictographs
            "\U0001F680-\U0001F6FF"  # transport & map symbols
            "\U0001F700-\U0001F77F"  # alchemical symbols
            "\U0001F780-\U0001F7FF"  # Geometric Shapes Extended
            "\U0001F800-\U0001F8FF"  # Supplemental Arrows-C
            "\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
            "\U0001FA00-\U0001FA6F"  # Chess Symbols
            "\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
            "\U00002702-\U000027B0"  # Dingbats
            "\U000024C2-\U0001F251"
            "]+",
            flags=re.UNICODE
        )
        return emoji_pattern.sub(r'', text)
    else:
        return None  # Return None if text is None

# Apply the function to the 'message_text' column
df['Message'] = df['Message'].apply(remove_emojis)

# Display the updated DataFrame
print(df.head())

  Message
0        
1        
2        
3        
4        


In [8]:
# Function to check if a token is a phone number
def is_phone_number(token):
    # Regex pattern for Ethiopian phone numbers (e.g., +251911234567 or 0911234567)
    phone_pattern = re.compile(r"^(\+251|0)\d{9}$")
    return bool(phone_pattern.match(token))

# Function to label tokens
def label_tokens(text):
    tokens = text.split()  # Tokenize by splitting on spaces
    labels = []
    for token in tokens:
        # Rule for Location (Amharic and English)
        if token in ["በቦሌ", "አዲስ", "በሜካሶ", "አድራሻ	" "Addis", "Ababa", "Megenagna", "Lebu", "Mebrat", "Lafto", "Medhanialem",  "ሞል", "ፎቅ", "ቁጥር.214", "ለቡ",  "ቤተ/ክርስቲያን",  "#ዛም_ሞል ቁ.1",  "መዳህኒዓለም"]:
            labels.append((token, "B-LOC"))
        elif token in ["አበባ"]:
            labels.append((token, "I-LOC"))
        # Rule for Product (Amharic and English)
        elif token in ["አዲስ", "ቤት", "አበባ", "ምርት", "product", "New", "quality", "bezat"]:
            labels.append((token, "B-Product" if token in ["አዲስ", "product", "New", "260°c","brush", "silicon",	"ሙቀት"] else "I-Product"))
        # Rule for Price (Amharic and English)
        elif token.isdigit() and not is_phone_number(token):  # Check if it's not a phone number
            labels.append((token, "B-PRICE"))
        elif token in ["ብር", "birr", "እስከ","ዋጋ", "ዋጋ-550ብር"]:
            labels.append((token, "I-PRICE"))
        # Default label for other tokens
        else:
            labels.append((token, "O"))
    return labels

# Apply labeling to each message
labeled_data = []
for message in df["Message"]:
    labeled_data.extend(label_tokens(message))
    labeled_data.append(("", ""))  # Add blank line between messages

# Save to a file in CoNLL format
with open("labeled_data.txt", "w", encoding="utf-8") as f:
    for token, label in labeled_data:
        if token == "":
            f.write("\n")  # Blank line for separation
        else:
            f.write(f"{token} {label}\n")

In [9]:
# Apply labeling to each message
labeled_data = []
for message in df["Message"]:
    labeled_data.extend(label_tokens(message))
    labeled_data.append(("", ""))  # Add blank line between messages

# Save to a file in CoNLL format
with open("labeled_data.txt", "w", encoding="utf-8") as f:
    for token, label in labeled_data:
        if token == "":
            f.write("\n")  # Blank line for separation
        else:
            f.write(f"{token} {label}\n")

In [10]:
labeled_data = pd.read_csv('/content/labeled_data.txt', sep=" ", header=None)

In [11]:
labeled_data

,0,1
0,3pcs,O
1,silicon,O
2,brush,O
3,spatulas,O
4,እስከ,I-PRICE
...,...,...
174640,#በተሻለ_ጥራት_የቀረበ,O
174641,ያሉበት,O
174642,ድረስ,O
174643,በነፃ,O


In [12]:
pip install transformers

In [13]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
model = AutoModelForTokenClassification.from_pretrained("xlm-roberta-base")
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

## setting up training arguments

In [15]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [16]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset, DatasetDict
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [21]:
from datasets import Dataset
# Load the labeled data as a pandas DataFrame
labeled_data = pd.read_csv('/content/labeled_data.txt', sep=" ", header=None, names=["token", "label"])

# Convert pandas DataFrame to Hugging Face Dataset
dataset = DatasetDict({
    "train": Dataset.from_pandas(labeled_data),
})

# Function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["token"],
        padding="max_length",
        truncation=True,
        is_split_into_words=True,
        # Explicitly set max_length to 512 or the desired value
        max_length=512
    )

# Tokenize the dataset
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    # Remove any cached data from previous runs with different settings to ensure fresh tokenization
    remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/174645 [00:00<?, ? examples/s]

In [23]:
from datasets import Dataset
# Load the labeled data as a pandas DataFrame
labeled_data = pd.read_csv('/content/labeled_data.txt', sep=" ", header=None, names=["token", "label"])

# Convert pandas DataFrame to Hugging Face Dataset
# Split the data into train and validation sets (e.g., 80% train, 20% validation)
train_data = labeled_data.sample(frac=0.8, random_state=42)  # Use random_state for reproducibility
validation_data = labeled_data.drop(train_data.index)

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_data),
    "validation": Dataset.from_pandas(validation_data),  # Add validation dataset
})

# Function to tokenize the dataset
def tokenize_function(examples):
    return tokenizer(
        examples["token"],
        padding="max_length",
        truncation=True,
        is_split_into_words=True,
        # Explicitly set max_length to 512 or the desired value
        max_length=512
    )

# Tokenize the dataset
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    # Remove any cached data from previous runs with different settings to ensure fresh tokenization
    remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/139716 [00:00<?, ? examples/s]

Map:   0%|          | 0/34929 [00:00<?, ? examples/s]

## Fine-Tune the model

In [24]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
)
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


ValueError: not enough values to unpack (expected 2, got 1)

## Save the model

In [25]:
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")

('./fine-tuned-model/tokenizer_config.json',
 './fine-tuned-model/special_tokens_map.json',
 './fine-tuned-model/sentencepiece.bpe.model',
 './fine-tuned-model/added_tokens.json',
 './fine-tuned-model/tokenizer.json')

## Model Comparison and Selection

In [38]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.4 MB/s eta 0:00:00


In [59]:
import evaluate

# Load the seqeval metric
metric = evaluate.load("seqeval")

In [60]:
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments
import evaluate  # Use evaluate instead of datasets
import numpy as np

# Load pre-trained models
models = {
    "XLM-Roberta": "xlm-roberta-base",
    "DistilBERT": "distilbert-base-multilingual-cased",
    "mBERT": "bert-base-multilingual-cased"
}



In [61]:
# Function to fine-tune and evaluate a model
def fine_tune_and_evaluate(model_name, model_checkpoint, tokenized_datasets):
    model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    training_args = TrainingArguments(
        output_dir=f"./{model_name}",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
    )

    trainer.train()
    predictions = trainer.predict(tokenized_datasets["validation"])
    preds = np.argmax(predictions.predictions, axis=-1)
    metric = evaluate.load("seqeval")
    results = metric.compute(predictions=preds, references=predictions.label_ids)
    return results



In [49]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=40f1be04e6de57b74b153119d08aa022cdc29467fc0c373fbae8332223cc4670
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [58]:
# Compare results
for model_name, result in results.items():
    print(f"{model_name}: {result}")

In [63]:
!pip install seqeval
from transformers import AutoModelForTokenClassification, AutoTokenizer, Trainer, TrainingArguments
import evaluate  # Use evaluate instead of datasets
import numpy as np

# Load pre-trained models
models = {
    "XLM-Roberta": "xlm-roberta-base",
    "DistilBERT": "distilbert-base-multilingual-cased",
    "mBERT": "bert-base-multilingual-cased"
}

# Assuming 'label_list' contains all unique labels in your dataset
label_list = sorted(list(set(labeled_data['label'])))  # Get unique labels from your labeled data

# Function to fine-tune and evaluate a model
def fine_tune_and_evaluate(model_name, model_checkpoint, tokenized_datasets):
    # Define the label2id and id2label mappings
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {i: label for i, label in enumerate(label_list)}

    model = AutoModelForTokenClassification.from_pretrained(
        model_checkpoint, num_labels=len(label_list), label2id=label2id, id2label=id2label
    )
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

    def align_labels_with_tokens(labels, word_ids):
        """Align labels with corresponding tokens."""
        new_labels = []
        current_word = None
        for word_id in word_ids:
            if word_id != current_word:
                # Start of a new word!
                current_word = word_id
                if word_id is not None:  # If this is a real token, get the label
                    label = labels[word_id]
                else:  # Special token (CLS, SEP, PAD) - assign "O" label
                    label = "O"  # or -100 to ignore in loss calculation
                new_labels.append(label)
            elif word_id is None:
                # Special token inside a word, ignore it
                continue

        return new_labels

    # Update the tokenize function to include labels
    def tokenize_and_align_labels(examples):
        tokenized_inputs = tokenizer(
            examples["token"],
            padding="max_length",
            truncation=True,
            is_split_into_words=True,
            max_length=512
        )
        # Get word ids from the tokenizer output
        all_labels = examples["label"]
        tokenized_inputs["labels"] = [
            align_labels_with_tokens(labels, word_ids)
            for labels, word_ids in zip(all_labels, tokenized_inputs.word_ids())
        ]
        return tokenized_inputs

    # Tokenize the dataset with label alignment
    tokenized_datasets = dataset.map(
        tokenize_and_align_labels,  # Use the updated tokenize function
        batched=True,
        remove_columns=dataset["train"].column_names
    )

    # Convert labels to numerical IDs using label2id
    def encode_labels(examples):
        examples["labels"] = [[label2id[label] for label in labels] for labels in examples["labels"]]
        return examples

    tokenized_datasets = tokenized_datasets.map(encode_labels, batched=True)

    training_args = TrainingArguments(
        output_dir=f"./{model_name}",
        evaluation_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
    )

    trainer.train()
    predictions = trainer.predict(tokenized_datasets["validation"])
    preds = np.argmax(predictions.predictions, axis=-1)

    # Convert predictions and references back to labels for seqeval
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, predictions.label_ids)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(preds, predictions.label_ids)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results

In [67]:
!pip install lime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 19.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=b3b2c2da7c6459347a920a611c362d6438fdb6dc22c792244b71cd16b4c29c7e
  Stored in directory: /root/.cache/pip/wheels/85/fa/a3/9c2d44c9f3cd77cf4e533b58900b2bf4487f2a17e8ec212a3d
Successfully built lime
